# Step1: Collect Images For Face Recognition

### Collecting Train Images

In [12]:
import cv2
import numpy as np


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


def face_extractor(img):
  
    
    faces = face_classifier.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        x=x-10
        y=y-10
        cropped_face = img[y:y+h+50, x:x+w+50]

    return cropped_face


cap = cv2.VideoCapture(0)
count = 0


while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (400, 400))
       
  #      file_name_path = 'C:/Users/Admin/Desktop/Face_Recognition/Deep_Learning_Face_Recognition/Datasets/Train/Pratik/' + str(count) + '.jpg'
        file_name_path = 'C:/Users/Admin/Desktop/Face_Recognition/Deep_Learning_Face_Recognition/Datasets/Train/Rushi/' + str(count) + '.jpg'
  
        cv2.imwrite(file_name_path, face)

       
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        #print("Face not found")
        pass

    if cv2.waitKey(1) == 1 or count == 200: 
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")


In [7]:
cap.release()
cv2.destroyAllWindows()     

### Collecting Test Images

In [13]:
import cv2
import numpy as np


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


def face_extractor(img):
  
    
    faces = face_classifier.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        x=x-10
        y=y-10
        cropped_face = img[y:y+h+50, x:x+w+50]

    return cropped_face


cap = cv2.VideoCapture(0)
count = 0


while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (400, 400))
       
      #  file_name_path = 'C:/Users/Admin/Desktop/Face_Recognition/Deep_Learning_Face_Recognition/Datasets/Test/Pratik/' + str(count) + '.jpg'
        file_name_path = 'C:/Users/Admin/Desktop/Face_Recognition/Deep_Learning_Face_Recognition/Datasets/Test/Rushi/' + str(count) + '.jpg'
  
        cv2.imwrite(file_name_path, face)

       
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        #print("Face not found")
        pass

    if cv2.waitKey(1) == 1 or count == 100: 
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")


# Step2 :Model Building and Training

In [14]:
import warnings
warnings.filterwarnings("ignore")
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [15]:
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/Train'
valid_path = 'Datasets/Test'


In [16]:

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  

In [17]:

folders = glob('Datasets/Train/*')
  

x = Flatten()(vgg.output)

# x = Dense(1000, activation='relu')(x)       for imagenet
prediction = Dense(len(folders), activation='softmax')(x)


model = Model(inputs=vgg.input, outputs=prediction)


model.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [18]:
model.compile(
  loss='categorical_crossentropy',   
  optimizer='adam',
  metrics=['accuracy']
)
 #categorical_crossentropy

In [19]:

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Datasets/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

'''r=model.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 5,
                         validation_data = test_set,
                         nb_val_samples = 2000)'''


Found 400 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


'r=model.fit_generator(training_set,\n                         samples_per_epoch = 8000,\n                         nb_epoch = 5,\n                         validation_data = test_set,\n                         nb_val_samples = 2000)'

In [20]:
len(test_set)

7

In [21]:
#len(training_set)
STEP_SIZE_VALID=test_set.n//test_set.batch_size

In [22]:
STEP_SIZE_VALID

6

In [23]:

# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=STEP_SIZE_VALID
)


Instructions for updating:
Use tf.cast instead.
Epoch 1/5
13/13 [==============================] - 693s 53s/step - loss: 0.5719 - accuracy: 0.7650 - val_loss: 0.0151 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 680s 52s/step - loss: 0.0360 - accuracy: 0.9875 - val_loss: 0.0130 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 627s 48s/step - loss: 5.2100e-04 - accuracy: 1.0000 - val_loss: 0.1821 - val_accuracy: 0.8869
Epoch 4/5
13/13 [==============================] - 624s 48s/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0987 - val_accuracy: 0.9107
Epoch 5/5
13/13 [==============================] - 630s 48s/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.1926 - val_accuracy: 0.8750


### Note : As the images are collected from webcam on famre basis without much change we see out model converge faster and as better accuracy in 2 epochs.

In [13]:
import tensorflow as tf

from keras.models import load_model

model.save('Face_features.h5')


# Step3: Recognizing faces 

In [2]:

import warnings
warnings.filterwarnings("ignore")

from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.models import load_model
import numpy as np



Using TensorFlow backend.


In [3]:
from keras.preprocessing import image
model = load_model('Face_features.h5')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
import cv2
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


In [5]:

def face_extractor(img):
  
    faces = face_cascade.detectMultiScale(img, 1.4,minNeighbors=2,minSize=(10,10))
    
    if faces is ():
        return None
   
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face


video_capture = cv2.VideoCapture(0)


while True:
    _, frame = video_capture.read()
   
   # frame=cv2.imread('Virushka1.jpg')
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
           
        img_array = np.array(im)
                    
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
       
        
        name="None matching"
        
      
        if(pred[0][0]>0.5):
            name1='Pratik'
            cv2.putText(frame,name1, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        elif(pred[0][1]>0.5):
            name2='Rushi'
            cv2.putText(frame,name2, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        else:
            cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



In [26]:
video_capture.release()
cv2.destroyAllWindows()